In [ ]:
import pycuba
import numpy as np

In [ ]:
pycuba.demo()

In [ ]:
import sys
from types import FunctionType
from copy import deepcopy

def get_dim(x_min,x_max):
    dim = len(x_min)
    if dim != len(x_max):
        raise Exception('The dimensions of x_min and x_max have to be the same!')
    return dim

def get_unitcube_func(func,x_min,x_max,dim=None):
    if not dim:
        dim = get_dim(x_min,x_max)
    x_min = np.array(x_min)
    x_max = np.array(x_max)
    min_functions=[]
    max_functions=[]
    for i in range(dim):
        if isinstance(x_min[i], FunctionType):
            min_functions.append(i)
        if isinstance(x_max[i], FunctionType):
            max_functions.append(i)
    def new_func(y):
        y = np.array([y[i] for i in range(dim)])
        if not min_functions and not max_functions:
            x_diff = x_max-x_min
            x = x_min+x_diff*y
        else:
            x_min_val = deepcopy(x_min)
            x_max_val = deepcopy(x_max)
            x_diff = np.full(dim, None)
            x = np.full(dim, None)
            for i in reversed(range(dim)): 
                if i in min_functions:
                    x_min_val[i] = x_min[i](x)
                if i in max_functions:
                    x_max_val[i] = x_max[i](x)
                x_diff[i] = x_max_val[i]-x_min_val[i]
                x[i] = x_min_val[i]+x_diff[i]*y[i]
            if np.isnan(x).any():
                sys.exit('There is an error in the functions specifying the limits of the integral!')
        jacobian = np.prod(x_diff)
        return func(x)*jacobian
    return new_func

def get_integrand(func,x_min,x_max,dim=None):
    if not dim:
        dim = get_dim(x_min,x_max)
    new_func = get_unitcube_func(func,x_min,x_max,dim=dim)
    def integrand(ndim,yy,ncomp,ff,userdata):
        ff[0] = new_func(yy)
        return 0
    return integrand

In [ ]:
def limit_func(x):
    return 3.*x[1]+3.

In [ ]:
def func(x_vec):
        x = x_vec[0]
        y = x_vec[1]
        return x**2/(np.cos(x + y + 1) + 5)
    
integrand = get_integrand(func,[1,2],[limit_func,7])

In [ ]:
def get_arguments_generic(function,x_min,x_max,
                         PrecisionGoal,AccuracyGoal,MinPoints,MaxPoints,
                         Verbose,Final,PseudoRandom,PseudoRandomSeed,StateFile,RetainStateFile):
    
    dim = get_dim(x_min,x_max)
    integrand = get_integrand(function,x_min,x_max,dim=dim)
    epsrel=10**-PrecisionGoal
    epsabs=10**-AccuracyGoal
    mineval=MinPoints
    maxeval=MaxPoints
    
    if Verbose not in [0,1,2,3]:
        raise Exception('Option "Verbose" has to be an Integer between 0 and 3.')
    flags = Verbose
    if Final != 'All':
        if Final == 'Last':
            flags = flags|2**2
        else:
            raise Exception('Option "Final" takes values "All" oder "Last".')
    # Bit 3 vegas and suave only
    if RetainStateFile is not False:
        if RetainStateFile is True:
            flags = flags|2**4
        else:
            raise Exception('Option "RetainStateFile" takes values "True" oder "False".')
    # Bit 5 vegas only
    if PseudoRandomSeed == 0:
        if PseudoRandom is not False:
            raise Exception('Option "PseudoRandom" has to be set to "False" if "PseudoRandomSeed" is set to "0".')
    elif (PseudoRandomSeed > 0 
    and isinstance(PseudoRandomSeed, int)
    and PseudoRandom >= 0
    and isinstance(PseudoRandom, int)):
        if PseudoRandom is True:
            PseudoRandom = 0
        elif PseudoRandom is False:
            PseudoRandomSeed = 0
        flags = flags|(2**8)*PseudoRandom
    else:
        raise Exception('Option "PseudoRandom" has to be set to "True", "False" or to a positive integer. "PseudoRandomSeed" has to be set to a positive integer.')
    seed = PseudoRandomSeed
    statefile = StateFile.encode('utf-8')

    
    return (integrand,dim,epsrel,epsabs,mineval,maxeval,flags,seed,statefile)

In [ ]:
def Divonne(function,x_min,x_max,
            PrecisionGoal=3,AccuracyGoal=12,MinPoints=0,MaxPoints=50000,
            Verbose=1,Final='All',PseudoRandom=False,
            PseudoRandomSeed=0,StateFile='',RetainStateFile=False,
            Key1=47,Key2=1,Key3=1,MaxPass=5,Border=0.,MaxChisq=10.,MinDeviation=.25):
    (integrand,dim,epsrel,epsabs,mineval,maxeval,flags,seed,statefile
    ) = get_arguments_generic(function,x_min,x_max,
                         PrecisionGoal,AccuracyGoal,MinPoints,MaxPoints,
                         Verbose,Final,PseudoRandom,PseudoRandomSeed,StateFile,RetainStateFile)
    
    return_dict = pycuba.Divonne(integrand, dim,
        epsrel=epsrel, epsabs=epsabs,
        mineval=mineval, maxeval=maxeval,
        key1=Key1, key2=Key2, key3=Key3, maxpass=MaxPass,
        border=Border, maxchisq=MaxChisq, mindeviation=MinDeviation, verbose=flags,
        statefile = statefile, seed=seed)
    
    return return_dict

In [ ]:
Divonne(func,[1,2],[limit_func,7],PrecisionGoal=6,MaxPoints=150000)

In [ ]:
Divonne(func,[1,2],[limit_func,7],PrecisionGoal=5,MaxPoints=150000,StateFile='teststate',RetainStateFile=False)

In [ ]:
NDIM = 2
MINEVAL = 0
MAXEVAL = 50000
KEY1 = 47
KEY2 = 1
KEY3 = 1
MAXPASS = 5
BORDER = 0.
MAXCHISQ = 10.
MINDEVIATION = .25
LDXGIVEN = NDIM
pycuba.Divonne(integrand, NDIM,
    mineval=MINEVAL, maxeval=MAXEVAL,
    key1=KEY1, key2=KEY2, key3=KEY3, maxpass=MAXPASS,
    border=BORDER, maxchisq=MAXCHISQ, mindeviation=MINDEVIATION,
    ldxgiven=LDXGIVEN, verbose=2)

In [ ]:
NDIM = 2
KEY = 0
pycuba.Cuhre(integrand, NDIM, key=KEY, verbose=2 | 4)